In [ ]:
import requests

url="https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=DE19346C5C8614F937946F0E3813F399"
url1 = 'https://bankrot.fedresurs.ru/DebtorsSearch.aspx?attempt=1'
session = requests.Session()
n_session = session.post(url1)
response = session.get(url)
print(response.text)

In [ ]:
from bs4 import BeautifulSoup
import re
import pyodbc
import openpyxl
import requests
import urllib
from urllib.parse import quote_plus as qp
from requests.utils import dict_from_cookiejar as dfc
import pandas as pd
import threading
from itertools import zip_longest
from math import ceil
import multiprocessing as mp
import time


connection_string = "Driver={SQL Server Native Client 11.0};Server=DESKTOP-R9VI2A2\SQLEXPRESS;Database=fedresurs;Trusted_Connection=yes;"
url1 = 'https://bankrot.fedresurs.ru/DebtorsSearch.aspx?attempt=1'
url2 ='https://bankrot.fedresurs.ru/DebtorsSearch.aspx'

class DatabaseWorker:
    def insert_in_debtor(self,debtor_firstname,debtor_middlename,debtor_lastname):
        sqlRequest = """INSERT INTO debtor (debtor_firstname,debtor_middlename,debtor_lastname,debtor_status)
                        VALUES(?,?,?,0)"""
        connection = pyodbc.connect(connection_string)
        dbCursor = connection.cursor()
        dbCursor.execute(sqlRequest,debtor_firstname,debtor_middlename,debtor_lastname)
        connection.commit()
        connection.close()

    def insert_debtor_info(self,debtor_id,debtor_category,debtor_inn,debtor_ogrnip,debtor_snils,debtor_region,debtor_address,debtor_link):
        sqlRequest = """
        INSERT INTO debtorInfo (debtor_id,debtor_category,debtor_inn,debtor_ogrnip,debtor_snils,debtor_region,debtor_address,debtor_link)
        VALUES(?,?,?, ?,?,?, ?,?)"""
        connection = pyodbc.connect(connection_string)
        dbCursor = connection.cursor()
        print("debtor_id: "+str(debtor_id))
        print("debtor_category: "+str(debtor_category))
        print("debtor_inn: "+str(debtor_inn))
        print("debtor_ogrnip: "+str(debtor_ogrnip))
        print("debtor_snils:" +str(debtor_snils))
        print("debtor_region: " + str(debtor_region))
        print("debtor_address: "+str(debtor_address))
        print("debtor_link:" +str(debtor_link))
        
        dbCursor.execute(sqlRequest,debtor_id,debtor_category,debtor_inn,debtor_ogrnip,debtor_snils,debtor_region,debtor_address,debtor_link)
        connection.commit()
        connection.close()

    def update_debtor(self,debtor_id):
        sqlRequest = """UPDATE debtor SET debtor_status=1 WHERE debtor_id=?"""
        connection = pyodbc.connect(connection_string)
        dbCursor = connection.cursor()
        dbCursor.execute(sqlRequest, debtor_id)
        connection.commit()
        connection.close()
    
    def search_unprocess_debtors(self):
        sqlRequest = "select debtor_id, debtor_firstname, debtor_middlename, debtor_lastname from debtor where debtor_status=0"
        connection = pyodbc.connect(connection_string)
        dbCursor = connection.cursor()
        dbCursor.execute(sqlRequest)
        cursorVocabulary = {}
        for row in dbCursor:
            cursorVocabulary[row.debtor_id] = [row.debtor_firstname, row.debtor_middlename, row.debtor_lastname]
        connection.commit()
        connection.close()
        return cursorVocabulary

class StringWorker:
    def search_debtor_link(self,td_with_link):
        result_url = "https://bankrot.fedresurs.ru"
        bs = BeautifulSoup(td_with_link,"html.parser")
        try:
            href = bs.findAll("a",href=True)[0]["href"]
            result_url += href
        except:
            print("При работе метода search_debtor_link произошла ошибка")
        finally:
            return result_url

    def preproc_line(self,line):
        line = re.sub("\n"," ",line)  #удаление переходов на новую строку
        line = re.sub("\t"," ",line) #удаление отступов (табуляции)
        line = re.sub("(\s+){2,}","",line) #удаление лишних пробелов
        return line

class Parser:
    def __init__(self):
        self.session = requests.Session()
        self.sw = StringWorker()
        self.db = DatabaseWorker()
        
    def write_debtor_info(self,debtor_id,response):
        soup = BeautifulSoup(response,"html.parser")
        searched_table = soup.find("table", attrs={ "id" : "ctl00_cphBody_gvDebtors"})
        
        soup2 = BeautifulSoup(str(searched_table),"html.parser")
        allTh = soup2.findAll("th")
        if len(allTh)>0:
            print("Всё ок")
            allTr = soup2.findAll("tr")
            for tr in allTr[1:]:
                debtor_category, debtor_link, debtor_region, debtor_address = "","","",""
                debtor_inn,debtor_ogrnip,debtor_snils = 0,0,0
                bs = BeautifulSoup(str(tr),"html.parser")
                allTd = bs.findAll("td")
                try:
                    debtor_category = self.sw.preproc_line(str(allTd[0].text)) # категория должника
                except:
                    debtor_category = "Category exists"
                try:
                    debtor_link = self.sw.search_debtor_link(str(allTd[1])) # ссылка на страницу с подробными данными должника
                except:
                    debtor_link = "Link exists"
                try:
                    debtor_inn = int(self.sw.preproc_line(str(allTd[2].text))) # ИНН должника
                except:
                    print("INN exists")
                try:
                    debtor_ogrnip = int(self.sw.preproc_line(str(allTd[3].text))) # ОРГНИП должника
                except:
                    print("ORGNIP exists")
                try:
                    debtor_snils = re.sub("\s","",str(allTd[4].text))
                    debtor_snils = int(re.sub("-","",debtor_snils))  # СНИЛС должника
                except:
                    print("SNILS exists")
                try:
                    debtor_region = self.sw.preproc_line(str(allTd[5].text)) # регион должника
                except:
                    debtor_region = "Region exists"
                try:
                    debtor_address = self.sw.preproc_line(str(allTd[6].text)) # полный адрес должника
                except:
                    debtor_address = "Address exists"
                self.db.insert_debtor_info(debtor_id,debtor_category,debtor_inn,debtor_ogrnip,debtor_snils,debtor_region,debtor_address,debtor_link)
                self.db.update_debtor(debtor_id)
        else:
            print("Пустая таблица")
            self.db.update_debtor(debtor_id)

                
    def write_debtors(self,filePath):
        excel_data_df = pd.read_excel(filePath, sheet_name='Sheet1')
        consumers = excel_data_df['Табельный номер'].tolist()
        for consumer in consumers:
            FIO = consumer.split(" ")
            try:
                last_name = FIO[0]
                first_name = FIO[1]
                middle_name = FIO[2]
                print("Фамилия: "+last_name+" Имя: " + first_name+" Отчество: "+ middle_name)
                self.db.insert_in_debtor(first_name,middle_name,last_name)
            except:
                print("Произошла ошибка")
                continue

    def search_debtors(self):
        threads = []
        debtors_list = self.db.search_unprocess_debtors()
        chunks = [debtors_list.items()]*2
        g = list(dict(filter(None,v)) for v in zip_longest(*chunks))
        part_len = ceil(len(g)/4)
        parting_list = [g[part_len*k:part_len*(k+1)] for k in range(4)]
        for part in parting_list:
            threads.append(threading.Thread(target=self.fetch_requests, args=(part,)))
            print(len(part))
        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()
    
    def fetch_requests(self,list_part):
        cookie_vocabulary ={}
        for word in list_part:
            for key in word.keys():#попробовать получать по индексу
                first_name = word[key][0]
                last_name = word[key][2]
                middle_name = word[key][1]
                debtorsearch = 'typeofsearch=Persons&orgname=&orgaddress=&orgregionid=&orgogrn=&orginn=&orgokpo=&OrgCategory=&prslastname='+qp(last_name)+'&prsfirstname='+qp(first_name)+'&prsmiddlename='+qp(middle_name)+'&prsaddress=&prsregionid=&prsinn=&prsogrn=&prssnils=&PrsCategory=&pagenumber=0'
                cookie_vocabulary[key]=debtorsearch
        n_session = self.session.post(url1)
        session_dictionary = dfc(n_session.cookies)
        for debtor_key in cookie_vocabulary.keys():
            dict2 = {}
            session_dictionary["debtorsearch"]=cookie_vocabulary[debtor_key]
            dict2["Cookie"] = "; ".join([str(x)+"="+str(y) for x,y in session_dictionary.items()])
            response = self.session.post(url2, headers=dict2)
            if response.status_code==200:
                self.write_debtor_info(debtor_key, str(response.text))
            else:
                print("Response code: "+str(response.status_code))

    
if __name__=="__main__":
    
    parser = Parser()
    db2 = DatabaseWorker()
    print("Выберите желаемое действие:")
    print("1 - Считать имена сотрудников для проверки")
    print("2 - Произвести поиск сотрудников")
    choice = int(input())
    if choice==1:
        parser.write_debtors("debtors2.xlsx")
    elif choice==2:
        threads = []
        debtors_list = db2.search_unprocess_debtors()
        chunks = [debtors_list.items()]*2
        g = list(dict(filter(None,v)) for v in zip_longest(*chunks))
        part_len = ceil(len(g)/5)
        parting_list = [g[part_len*k:part_len*(k+1)] for k in range(5)]
        for part in parting_list:
            threads.append(threading.Thread(target=parser.fetch_requests, args=(part,)))
            print(len(part))
        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()
        #parser.search_debtors()
    else:
        print("Вы ввели неверный параметр")

In [ ]:
from bs4 import BeautifulSoup
import re
import pyodbc
import openpyxl
import requests
import urllib
from urllib.parse import quote_plus as qp
from requests.utils import dict_from_cookiejar as dfc
import pandas as pd
import threading
from itertools import zip_longest
from math import ceil
import multiprocessing as mp
import time


connection_string = "Driver={SQL Server Native Client 11.0};Server=DESKTOP-R9VI2A2\SQLEXPRESS;Database=fedresurs;Trusted_Connection=yes;"
url1 = 'https://bankrot.fedresurs.ru/DebtorsSearch.aspx?attempt=1'
url2 ='https://bankrot.fedresurs.ru/DebtorsSearch.aspx'

class DatabaseWorker:
    def insert_in_debtor(self,debtor_firstname,debtor_middlename,debtor_lastname):
        sqlRequest = """INSERT INTO debtor (debtor_firstname,debtor_middlename,debtor_lastname,debtor_status)
                        VALUES(?,?,?,0)"""
        connection = pyodbc.connect(connection_string)
        dbCursor = connection.cursor()
        dbCursor.execute(sqlRequest,debtor_firstname,debtor_middlename,debtor_lastname)
        connection.commit()
        connection.close()

    def insert_debtor_info(self,debtor_id,debtor_category,debtor_inn,debtor_ogrnip,debtor_snils,debtor_region,debtor_address,debtor_link):
        sqlRequest = """
        INSERT INTO debtorInfo (debtor_id,debtor_category,debtor_inn,debtor_ogrnip,debtor_snils,debtor_region,debtor_address,debtor_link)
        VALUES(?,?,?, ?,?,?, ?,?)"""
        connection = pyodbc.connect(connection_string)
        dbCursor = connection.cursor()
        print("debtor_id: "+str(debtor_id))
        print("debtor_category: "+str(debtor_category))
        print("debtor_inn: "+str(debtor_inn))
        print("debtor_ogrnip: "+str(debtor_ogrnip))
        print("debtor_snils:" +str(debtor_snils))
        print("debtor_region: " + str(debtor_region))
        print("debtor_address: "+str(debtor_address))
        print("debtor_link:" +str(debtor_link))
        
        dbCursor.execute(sqlRequest,debtor_id,debtor_category,debtor_inn,debtor_ogrnip,debtor_snils,debtor_region,debtor_address,debtor_link)
        connection.commit()
        connection.close()

    def update_debtor(self,debtor_id):
        sqlRequest = """UPDATE debtor SET debtor_status=1 WHERE debtor_id=?"""
        connection = pyodbc.connect(connection_string)
        dbCursor = connection.cursor()
        dbCursor.execute(sqlRequest, debtor_id)
        connection.commit()
        connection.close()
    
    def search_unprocess_debtors(self):
        sqlRequest = "select debtor_id, debtor_firstname, debtor_middlename, debtor_lastname from debtor where debtor_status=0"
        connection = pyodbc.connect(connection_string)
        dbCursor = connection.cursor()
        dbCursor.execute(sqlRequest)
        cursorVocabulary = {}
        for row in dbCursor:
            cursorVocabulary[row.debtor_id] = [row.debtor_firstname, row.debtor_middlename, row.debtor_lastname]
        connection.commit()
        connection.close()
        return cursorVocabulary

class StringWorker:
    def search_debtor_link(self,td_with_link):
        result_url = "https://bankrot.fedresurs.ru"
        bs = BeautifulSoup(td_with_link,"html.parser")
        try:
            href = bs.findAll("a",href=True)[0]["href"]
            result_url += href
        except:
            print("При работе метода search_debtor_link произошла ошибка")
        finally:
            return result_url

    def preproc_line(self,line):
        line = re.sub("\n"," ",line)  #удаление переходов на новую строку
        line = re.sub("\t"," ",line) #удаление отступов (табуляции)
        line = re.sub("(\s+){2,}","",line) #удаление лишних пробелов
        return line

class Parser:
    def __init__(self):
        self.session = requests.Session()
        self.sw = StringWorker()
        self.db = DatabaseWorker()
        
    def write_debtor_info(self,debtor_id,response):
        soup = BeautifulSoup(response,"html.parser")
        searched_table = soup.find("table", attrs={ "id" : "ctl00_cphBody_gvDebtors"})
        
        soup2 = BeautifulSoup(str(searched_table),"html.parser")
        allTh = soup2.findAll("th")
        if len(allTh)>0:
            print("Всё ок")
            allTr = soup2.findAll("tr")
            for tr in allTr[1:]:
                debtor_category, debtor_link, debtor_region, debtor_address = "","","",""
                debtor_inn,debtor_ogrnip,debtor_snils = 0,0,0
                bs = BeautifulSoup(str(tr),"html.parser")
                allTd = bs.findAll("td")
                try:
                    debtor_category = self.sw.preproc_line(str(allTd[0].text)) # категория должника
                except:
                    debtor_category = "Category exists"
                try:
                    debtor_link = self.sw.search_debtor_link(str(allTd[1])) # ссылка на страницу с подробными данными должника
                except:
                    debtor_link = "Link exists"
                try:
                    debtor_inn = int(self.sw.preproc_line(str(allTd[2].text))) # ИНН должника
                except:
                    print("INN exists")
                try:
                    debtor_ogrnip = int(self.sw.preproc_line(str(allTd[3].text))) # ОРГНИП должника
                except:
                    print("ORGNIP exists")
                try:
                    debtor_snils = re.sub("\s","",str(allTd[4].text))
                    debtor_snils = int(re.sub("-","",debtor_snils))  # СНИЛС должника
                except:
                    print("SNILS exists")
                try:
                    debtor_region = self.sw.preproc_line(str(allTd[5].text)) # регион должника
                except:
                    debtor_region = "Region exists"
                try:
                    debtor_address = self.sw.preproc_line(str(allTd[6].text)) # полный адрес должника
                except:
                    debtor_address = "Address exists"
                self.db.insert_debtor_info(debtor_id,debtor_category,debtor_inn,debtor_ogrnip,debtor_snils,debtor_region,debtor_address,debtor_link)
                self.db.update_debtor(debtor_id)
        else:
            print("Пустая таблица")
            self.db.update_debtor(debtor_id)

                
    def write_debtors(self,filePath):
        excel_data_df = pd.read_excel(filePath, sheet_name='Sheet1')
        consumers = excel_data_df['Табельный номер'].tolist()
        for consumer in consumers:
            FIO = consumer.split(" ")
            try:
                last_name = FIO[0]
                first_name = FIO[1]
                middle_name = FIO[2]
                print("Фамилия: "+last_name+" Имя: " + first_name+" Отчество: "+ middle_name)
                self.db.insert_in_debtor(first_name,middle_name,last_name)
            except:
                print("Произошла ошибка")
                continue

    def search_debtors(self):
        threads = []
        debtors_list = self.db.search_unprocess_debtors()
        chunks = [debtors_list.items()]*2
        g = list(dict(filter(None,v)) for v in zip_longest(*chunks))
        part_len = ceil(len(g)/4)
        parting_list = [g[part_len*k:part_len*(k+1)] for k in range(4)]
        for part in parting_list:
            threads.append(threading.Thread(target=self.fetch_requests, args=(part,)))
            print(len(part))
        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()
    
    def fetch_requests(self,list_part):
        cookie_vocabulary ={}
        n_session = self.session.post(url1)
        session_dictionary = dfc(n_session.cookies)
        for word in list_part:
            for key in word.keys():#попробовать получать по индексу
                first_name = word[key][0]
                last_name = word[key][2]
                middle_name = word[key][1]
                debtorsearch = 'typeofsearch=Persons&orgname=&orgaddress=&orgregionid=&orgogrn=&orginn=&orgokpo=&OrgCategory=&prslastname='+qp(last_name)+'&prsfirstname='+qp(first_name)+'&prsmiddlename='+qp(middle_name)+'&prsaddress=&prsregionid=&prsinn=&prsogrn=&prssnils=&PrsCategory=&pagenumber=0'
                dict2 = {}
                session_dictionary["debtorsearch"]=debtorsearch
                dict2["Cookie"] = "; ".join([str(x)+"="+str(y) for x,y in session_dictionary.items()])
                response = self.session.post(url2, headers=dict2)
                if response.status_code==200:
                    self.write_debtor_info(key, str(response.text))
                else:
                    print("Response code: "+str(response.status_code))
    
if __name__=="__main__":
    
    parser = Parser()
    db2 = DatabaseWorker()
    print("Выберите желаемое действие:")
    print("1 - Считать имена сотрудников для проверки")
    print("2 - Произвести поиск сотрудников")
    choice = int(input())
    if choice==1:
        parser.write_debtors("debtors2.xlsx")
    elif choice==2:
        threads = []
        debtors_list = db2.search_unprocess_debtors()
        chunks = [debtors_list.items()]*2
        g = list(dict(filter(None,v)) for v in zip_longest(*chunks))
        part_len = ceil(len(g)/2)
        parting_list = [g[part_len*k:part_len*(k+1)] for k in range(2)]
        for part in parting_list:
            threads.append(threading.Thread(target=parser.fetch_requests, args=(part,)))
            print(len(part))
        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()
        #parser.search_debtors()
    else:
        print("Вы ввели неверный параметр")

In [1]:
from bs4 import BeautifulSoup
import re
import pyodbc
import openpyxl
import requests
import urllib
from urllib.parse import quote_plus as qp
from requests.utils import dict_from_cookiejar as dfc
import pandas as pd
import threading
from itertools import zip_longest
from math import ceil
import multiprocessing as mp
import time


connection_string = "Driver={SQL Server Native Client 11.0};Server=DESKTOP-R9VI2A2\SQLEXPRESS;Database=fedresurs;Trusted_Connection=yes;"
url1 = 'https://bankrot.fedresurs.ru/DebtorsSearch.aspx?attempt=1'
url2 ='https://bankrot.fedresurs.ru/DebtorsSearch.aspx'

class DatabaseWorker:
    def insert_in_debtor(self,debtor_firstname,debtor_middlename,debtor_lastname):
        sqlRequest = """INSERT INTO debtor (debtor_firstname,debtor_middlename,debtor_lastname,debtor_status)
                        VALUES(?,?,?,0)"""
        connection = pyodbc.connect(connection_string)
        dbCursor = connection.cursor()
        dbCursor.execute(sqlRequest,debtor_firstname,debtor_middlename,debtor_lastname)
        connection.commit()
        connection.close()

    def insert_debtor_info(self,debtor_id,debtor_category,debtor_inn,debtor_ogrnip,debtor_snils,debtor_region,debtor_address,debtor_link):
        sqlRequest = """
        INSERT INTO debtorInfo (debtor_id,debtor_category,debtor_inn,debtor_ogrnip,debtor_snils,debtor_region,debtor_address,debtor_link)
        VALUES(?,?,?, ?,?,?, ?,?)"""
        connection = pyodbc.connect(connection_string)
        dbCursor = connection.cursor()
        print("debtor_id: "+str(debtor_id))
        print("debtor_category: "+str(debtor_category))
        print("debtor_inn: "+str(debtor_inn))
        print("debtor_ogrnip: "+str(debtor_ogrnip))
        print("debtor_snils:" +str(debtor_snils))
        print("debtor_region: " + str(debtor_region))
        print("debtor_address: "+str(debtor_address))
        print("debtor_link:" +str(debtor_link))
        dbCursor.execute(sqlRequest,debtor_id,debtor_category,debtor_inn,debtor_ogrnip,debtor_snils,debtor_region,debtor_address,debtor_link)
        connection.commit()
        connection.close()

    def update_debtor(self,debtor_id):
        sqlRequest = """UPDATE debtor SET debtor_status=1 WHERE debtor_id=?"""
        connection = pyodbc.connect(connection_string)
        dbCursor = connection.cursor()
        dbCursor.execute(sqlRequest, debtor_id)
        connection.commit()
        connection.close()
    
    def search_unprocess_debtors(self):
        sqlRequest = "select debtor_id, debtor_firstname, debtor_middlename, debtor_lastname from debtor where debtor_status=0"
        connection = pyodbc.connect(connection_string)
        dbCursor = connection.cursor()
        dbCursor.execute(sqlRequest)
        cursorVocabulary = {}
        for row in dbCursor:
            cursorVocabulary[row.debtor_id] = [row.debtor_firstname, row.debtor_middlename, row.debtor_lastname]
        connection.commit()
        connection.close()
        return cursorVocabulary

class StringWorker:
    def search_debtor_link(self,td_with_link):
        result_url = "https://bankrot.fedresurs.ru"
        bs = BeautifulSoup(td_with_link,"html.parser")
        try:
            href = bs.findAll("a",href=True)[0]["href"]
            result_url += href
        except:
            print("При работе метода search_debtor_link произошла ошибка")
        finally:
            return result_url

    def preproc_line(self,line):
        line = re.sub("\n"," ",line)  #удаление переходов на новую строку
        line = re.sub("\t"," ",line) #удаление отступов (табуляции)
        line = re.sub("(\s+){2,}","",line) #удаление лишних пробелов
        return line

class Parser:
    def __init__(self):
        self.session = requests.Session()
        self.sw = StringWorker()
        self.db = DatabaseWorker()
        
    #def write_debtor_info(self,debtor_id,response):
    
    def write_debtors(self,filePath):
        excel_data_df = pd.read_excel(filePath, sheet_name='Sheet1')
        consumers = excel_data_df['Табельный номер'].tolist()
        for consumer in consumers:
            FIO = consumer.split(" ")
            try:
                last_name = FIO[0]
                first_name = FIO[1]
                middle_name = FIO[2]
                print("Фамилия: "+last_name+" Имя: " + first_name+" Отчество: "+ middle_name)
                self.db.insert_in_debtor(first_name,middle_name,last_name)
            except:
                print("Произошла ошибка")
                continue

    def search_debtors(self):
        threads = []
        debtors_list = self.db.search_unprocess_debtors()
        chunks = [debtors_list.items()]*2
        g = list(dict(filter(None,v)) for v in zip_longest(*chunks))
        part_len = ceil(len(g)/4)
        parting_list = [g[part_len*k:part_len*(k+1)] for k in range(4)]
        for part in parting_list:
            threads.append(threading.Thread(target=self.fetch_requests, args=(part,)))
            print(len(part))
        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()
    
    def fetch_requests(self,list_part):
        cookie_vocabulary ={}
        n_session = self.session.post(url1)
        session_dictionary = dfc(n_session.cookies)
        for word in list_part:
            for key in word.keys():#попробовать получать по индексу
                first_name = word[key][0]
                last_name = word[key][2]
                middle_name = word[key][1]
                debtorsearch = 'typeofsearch=Persons&orgname=&orgaddress=&orgregionid=&orgogrn=&orginn=&orgokpo=&OrgCategory=&prslastname='+qp(last_name)+'&prsfirstname='+qp(first_name)+'&prsmiddlename='+qp(middle_name)+'&prsaddress=&prsregionid=&prsinn=&prsogrn=&prssnils=&PrsCategory=&pagenumber=0'
                dict2 = {}
                session_dictionary["debtorsearch"]=debtorsearch
                dict2["Cookie"] = "; ".join([str(x)+"="+str(y) for x,y in session_dictionary.items()])
                response = self.session.post(url2, headers=dict2)
                if response.status_code==200:
                    soup = BeautifulSoup(response.text,"html.parser")
                    searched_table = soup.find("table", attrs={ "id" : "ctl00_cphBody_gvDebtors"})
                    soup2 = BeautifulSoup(str(searched_table),"html.parser")
                    allTh = soup2.findAll("th")
                    if len(allTh)>0:
                        print("Всё ок")
                        allTr = soup2.findAll("tr")
                        for tr in allTr[1:]:
                            debtor_category, debtor_link, debtor_region, debtor_address = "","","",""
                            debtor_inn,debtor_ogrnip,debtor_snils = 0,0,0
                            bs = BeautifulSoup(str(tr),"html.parser")
                            allTd = bs.findAll("td")
                            try:
                                debtor_category = self.sw.preproc_line(str(allTd[0].text)) # категория должника
                            except:
                                debtor_category = "Category exists"
                            try:
                                debtor_link = self.sw.search_debtor_link(str(allTd[1])) # ссылка на страницу с подробными данными должника
                            except:
                                debtor_link = "Link exists"
                            try:
                                debtor_inn = int(self.sw.preproc_line(str(allTd[2].text))) # ИНН должника
                            except:
                                print("INN exists")
                            try:
                                debtor_ogrnip = int(self.sw.preproc_line(str(allTd[3].text))) # ОРГНИП должника
                            except:
                                print("ORGNIP exists")
                            try:
                                debtor_snils = re.sub("\s","",str(allTd[4].text))
                                debtor_snils = int(re.sub("-","",debtor_snils))  # СНИЛС должника
                            except:
                                print("SNILS exists")
                            try:
                                debtor_region = self.sw.preproc_line(str(allTd[5].text)) # регион должника
                            except:
                                debtor_region = "Region exists"
                            try:
                                debtor_address = self.sw.preproc_line(str(allTd[6].text)) # полный адрес должника
                            except:
                                debtor_address = "Address exists"
                            self.db.insert_debtor_info(key,debtor_category,debtor_inn,debtor_ogrnip,debtor_snils,debtor_region,debtor_address,debtor_link)
                            self.db.update_debtor(key)
                    else:
                        print("Пустая таблица")
                        self.db.update_debtor(key)
                else:
                    print("Response code: "+str(response.status_code))
    
if __name__=="__main__":
    
    parser = Parser()
    db2 = DatabaseWorker()
    print("Выберите желаемое действие:")
    print("1 - Считать имена сотрудников для проверки")
    print("2 - Произвести поиск сотрудников")
    choice = int(input())
    if choice==1:
        parser.write_debtors("debtors2.xlsx")
    elif choice==2:
        threads = []
        debtors_list = db2.search_unprocess_debtors()
        chunks = [debtors_list.items()]*2
        g = list(dict(filter(None,v)) for v in zip_longest(*chunks))
        part_len = ceil(len(g)/4)
        parting_list = [g[part_len*k:part_len*(k+1)] for k in range(4)]
        for part in parting_list:
            threads.append(threading.Thread(target=parser.fetch_requests, args=(part,)))
            print(len(part))
        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()
        #parser.search_debtors()
    else:
        print("Вы ввели неверный параметр")

Выберите желаемое действие:
1 - Считать имена сотрудников для проверки
2 - Произвести поиск сотрудников
2
3465
3465
3465
3465
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Всё ок
ORGNIP exists
debtor_id: 38816
debtor_category: Физическое лицо
debtor_inn: 781002424430
debtor_ogrnip: 0
debtor_snils:6816752288
debtor_region: г. Санкт-Петербург
debtor_address: г. Санкт-Петербург, ул. Летчика Пилютова, д. 32, кв. 6
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=79BD4210A7497E0906942057260BCD67
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Всё ок


Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Всё ок
ORGNIP exists
SNILS exists
debtor_id: 38877
debtor_category: Физическое лицо
debtor_inn: 772915551071
debtor_ogrnip: 0
debtor_snils:
debtor_region: г. Москва
debtor_address: 119501, г. Москва, ул. Веерная, д. 5, корп. 1, кв. 73
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=CBAFFB78E29EFF1BB554029CFF3F7440
Пустая таблица
Всё ок
ORGNIP exists
debtor_id: 45894
debtor_category: Физическое лицо
debtor_inn: 480204785969
debtor_ogrnip: 0
debtor_snils:7015909545
debtor_region: Лип

Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Всё ок
ORGNIP exists
debtor_id: 38936
debtor_category: Физическое лицо
debtor_inn: 381601666406
debtor_ogrnip: 0
debtor_snils:4799384123
debtor_region: Иркутская область
debtor_address: 665268, Иркутская область, г.Тулун, мкр.Угольщиков, д.8, кв.64
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=6E04F257BBA13D1B8C64C159D2478D0E
Всё ок
ORGNIP exists
debtor_id: 42493
debtor_category: Физическое лицо
debtor_inn: 781128991389
debtor_ogrnip: 0
debtor_snils:6296821893
debtor_region: г. Санкт-Петербург
debtor_address:

Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Всё ок
ORGNIP exists
debtor_id: 45909
debtor_category: Физическое лицо
debtor_inn: 540427359697
debtor_ogrnip: 0
debtor_snils:6694749525
debtor_region: Новосибирская область
debtor_address: Новосибирская область, г. Новосибирск, ул. Судоремонтная, д. 11
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=AEF3E705BDA30CAA32E45A6A19B7C29E
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Всё ок
debtor_id: 42383
debtor_category: Индивидуальный предприниматель
debtor_inn: 440201260686
debtor_ogrnip: 310760232900033
debtor_snils:10451517008
debtor_region: Вологодская область
debtor_address: г. Воло

ORGNIP exists
debtor_id: 42426
debtor_category: Физическое лицо
debtor_inn: 781701267082
debtor_ogrnip: 0
debtor_snils:6765782012
debtor_region: г. Санкт-Петербург
debtor_address: 196641, г. Санкт-Петербург, пос. Металлострой, ул. Полевая, д. 19, кв. 64
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=9E916AD260D577A8E834F67187B857DF
Пустая таблица
Пустая таблица
Всё ок
SNILS exists
debtor_id: 42427
debtor_category: Индивидуальный предприниматель
debtor_inn: 504200080364
debtor_ogrnip: 304504222400032
debtor_snils:
debtor_region: Московская область
debtor_address: 141300, Московская обл., г. Сергиев Посад-15,
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=4E7874B700F6817B53747C17E7977D40
Пустая таблица
ORGNIP exists
debtor_id: 42427
debtor_category: Физическое лицо
debtor_inn: 352528822531
debtor_ogrnip: 0
debtor_snils:13259122439
debtor_region: Вологодская область
debtor_address: 160022, г. Вологда, ул. Ярославская, д. 40а, кв. 8
debtor_link:https

...

Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Всё ок
ORGNIP exists
debtor_id: 45618
debtor_category: Физическое лицо
debtor_inn: 165043082181
debtor_ogrnip: 0
debtor_snils:5525027132
debtor_region: Республика Татарстан
debtor_address: Пестричинский р-н, с. Богородское, РТ
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=6479A8B853E1346AA8D4E69AA0596DB9
Пустая таблица
Пустая таблица
Всё ок
ORGNIP exists
debtor_id: 71940
debtor_category: Физическое лицо
debtor_inn: 772628675281
debtor_ogrnip: 0
debtor_snils:1946252051
debtor_region: г. Москва
debtor_address: 117525, г.Москва, ул.Чертановская,, д.16 корп.2 кв.215
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=30B5332ECB323AABC3E4AFEAB1BD5075
ORGNIP exists
debtor_id: 71940
debtor_category: Физическое лицо
debtor_inn: 503501444662
debtor_ogrnip: 0
debtor_snils:4626826671
debtor_region: Московская область
debtor_address: Московская область, гор. Павловский Поса

Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Всё ок
ORGNIP exists
debtor_id: 71996
debtor_category: Физическое лицо
debtor_inn: 780151513798
debtor_ogrnip: 0
debtor_snils:5217167232
debtor_region: г. Санкт-Петербург
debtor_address: 199397, Россия, Санкт-Петербург, ул. Кораблестроителей, 46,к.1,кв.616
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=BE3E2F5915E7928A8DE49434220A60B7
Пустая таблица
Пустая таблица
Пустая таблица
Всё ок
ORGNIP exists
debtor_id: 71997
debtor_category: Физическое лицо
debtor_inn: 165119182534
debtor_ogrnip: 0
debtor_snils:10642609632
debtor_region: Республика Татарстан
debtor_address: 423585, г. Нижнекамск, пр-кт Химиков 14б, кв. 56
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=63FB97210A28D1096FA4D44D11268DE5
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пуст

ORGNIP exists
debtor_id: 42134
debtor_category: Физическое лицо
debtor_inn: 622909499608
debtor_ogrnip: 0
debtor_snils:14644719782
debtor_region: Рязанская область
debtor_address: г.Рязань, ул.Московское шоссе, д.39, кв.94
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=F94383124CDE37BA0974FF8E39C89E8D
ORGNIP exists
debtor_id: 42134
debtor_category: Физическое лицо
debtor_inn: 891301645684
debtor_ogrnip: 0
debtor_snils:7741560172
debtor_region: Московская область
debtor_address: Московская обл, Балашиха г, Зеленая ул, дом № 25, квартира 168
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=DB5F878F9ACA326BB3A4C1932BB74D06
Пустая таблица
Пустая таблица
Пустая таблица
Всё ок
debtor_id: 42138
debtor_category: Крестьянское (фермерское) хозяйство
debtor_inn: 611500650072
debtor_ogrnip: 304611534300018
debtor_snils:7101500250
debtor_region: Ростовская область
debtor_address: 346200, Ростовская обл.. Кашарский р-н, х. Усиковка, ул. Береговая, 1
debtor_link:

ORGNIP exists
debtor_id: 72134
debtor_category: Физическое лицо
debtor_inn: 272514540907
debtor_ogrnip: 0
debtor_snils:11034207582
debtor_region: Хабаровский край
debtor_address: Хабаровский край, с.Мичуринское, ул. Новая д.6 кв.1
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=6C499174C26BB5C84974E21086ED9CE9
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Пустая таблица
Всё ок
ORGNIP exists
debtor_id: 42216
debtor_category: Физическое лицо
debtor_inn: 645400060100
debtor_ogrnip: 0
debtor_snils:5330512914
debtor_region: Саратовская область
debtor_address: Саратовская обл., г. Саратов, ул. Кутякова, д. 10, кв. 3
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=FAC6110D6EE7BB9950D4D314ACA4EC62
ORGNIP exists
SNILS exists
debtor_id: 42216
debtor_category: Физическое лицо
debtor_inn: 682400133278
debtor_ogrnip: 0
debtor_snils:
debtor_region: Тамбовская область
debtor_address: Тамбов-4, 23,21
debtor_link:https://bankrot.fedresurs.ru/PrivatePe

Пустая таблица
ORGNIP exists
debtor_id: 42243
debtor_category: Физическое лицо
debtor_inn: 615011045083
debtor_ogrnip: 0
debtor_snils:14859401493
debtor_region: Ростовская область
debtor_address: 346400, Ростовская область,, г.Новочеркасск, ул. Крылова, д. 5, кв. 12
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=967DC6901ACC9FCAB424F6E05D38EE66
ORGNIP exists
debtor_id: 42243
debtor_category: Физическое лицо
debtor_inn: 540864533370
debtor_ogrnip: 0
debtor_snils:9733935818
debtor_region: Новосибирская область
debtor_address: г. Новосибирск, ул. Демакова, д. 5, кв. 34
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=3FC78020F45962C87F144CC3D6595A57
ORGNIP exists
SNILS exists
debtor_id: 42243
debtor_category: Физическое лицо
debtor_inn: 712801272118
debtor_ogrnip: 0
debtor_snils:
debtor_region: Самарская область
debtor_address: Тульская область, г. Липки, ул. Жуковского, д. 30
debtor_link:https://bankrot.fedresurs.ru/PrivatePersonCard.aspx?ID=AD689489